In [3]:
import sys
from pathlib import Path
Path(sys.executable).as_posix()


'C:/Users/colli/anaconda3/python.exe'

In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px
import altair as alt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pmdarima as pmd

def arimamodel(timeseriesarray):
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              trace=True)
    return autoarima_model

In [3]:
df1 = pd.read_csv(r'TTS_LBNL_public_file_19-Oct-2020_p1.csv') 
df2 = pd.read_csv(r'TTS_LBNL_public_file_19-Oct-2020_p2.csv',encoding="ISO-8859-1") 
df = df1.append(df2)

In [6]:
tot_sales_by_state = pd.read_excel (r'../data/Sales to Ultimate Customers (Megawatthours) by State.xlsx')

In [7]:
avg_cost_by_state = pd.read_excel (r'../data/Average Price (Cents per kilowatthour) by State.xlsx')

In [4]:
install_df = df[['installation_date','system_size_DC','total_installed_price','rebate_or_grant','zip_code', 'city', 'state','utility_service_territory']]
install_df['installation_date'] = pd.to_datetime(install_df['installation_date'])
install_df['year'] = pd.DatetimeIndex(install_df['installation_date']).year
install_df = install_df[install_df.total_installed_price != -9999]
install_df = install_df.groupby(['year','state']).agg({'total_installed_price':'mean'})
install_df = pd.pivot_table(install_df, values='total_installed_price', index=['year'], columns=['state'])
install_df = install_df.apply(lambda x: x.fillna(x.mean()),axis=0)
install_df = install_df.drop(columns=['OR','DE'])

In [8]:
res_sales_df = pd.pivot_table(tot_sales_by_state, values='Residential', index=['State'], columns=['Year']).T
res_sales_df = res_sales_df.drop(columns=['US'])

In [9]:
res_avg_cost_df = pd.pivot_table(avg_cost_by_state, values='Residential', index=['State'], columns=['Year']).T
res_avg_cost_df = res_avg_cost_df.drop(columns=['US'])

In [10]:
def predict_install_cost():
    df = pd.DataFrame()
    states = install_df.columns

    for state in states:
        x = install_df[state]
        for i in range(11):
            model = ARIMA(x, order=(2,0,0))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x))
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={22: 2020}, inplace=True)
    df.rename(index={23: 2021}, inplace=True)
    df.rename(index={24: 2022}, inplace=True)
    df.rename(index={25: 2023}, inplace=True)
    df.rename(index={26: 2024}, inplace=True)
    df.rename(index={27: 2025}, inplace=True)
    df.rename(index={28: 2026}, inplace=True)
    df.rename(index={29: 2027}, inplace=True)
    df.rename(index={30: 2028}, inplace=True)
    df.rename(index={31: 2029}, inplace=True)
    df.rename(index={32: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank()
    df['Weight Score'] = df['Rank']*.3

    return df.sort_values('Rank', ascending=True).reset_index()

In [11]:
def predict_usage(sector_df):
    df = pd.DataFrame()
    states = sector_df.columns

    for state in states:
        x = sector_df[state]
        for i in range(11):
            model = ARIMA(x, order=(1, 2, 0))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x), typ='levels')
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={30: 2020}, inplace=True)
    df.rename(index={31: 2021}, inplace=True)
    df.rename(index={32: 2022}, inplace=True)
    df.rename(index={33: 2023}, inplace=True)
    df.rename(index={34: 2024}, inplace=True)
    df.rename(index={35: 2025}, inplace=True)
    df.rename(index={36: 2026}, inplace=True)
    df.rename(index={37: 2027}, inplace=True)
    df.rename(index={38: 2028}, inplace=True)
    df.rename(index={39: 2029}, inplace=True)
    df.rename(index={40: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank()
    df['Weight Score'] = df['Rank']*.2

    return df.sort_values('Rank', ascending=True).reset_index()

In [12]:
def predict_cost(sector_df):
    df = pd.DataFrame()
    states = sector_df.columns

    for state in states:
        x = sector_df[state]
        for i in range(11):
            model = ARIMA(x, order=(0,2,1))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x), typ='levels')
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={30: 2020}, inplace=True)
    df.rename(index={31: 2021}, inplace=True)
    df.rename(index={32: 2022}, inplace=True)
    df.rename(index={33: 2023}, inplace=True)
    df.rename(index={34: 2024}, inplace=True)
    df.rename(index={35: 2025}, inplace=True)
    df.rename(index={36: 2026}, inplace=True)
    df.rename(index={37: 2027}, inplace=True)
    df.rename(index={38: 2028}, inplace=True)
    df.rename(index={39: 2029}, inplace=True)
    df.rename(index={40: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank(ascending=False)
    df['Weight Score'] = df['Rank']*.5

    return df.sort_values('Rank', ascending=True).reset_index()

In [13]:
res_sales_prediction = predict_usage(res_sales_df)
res_cost_prediction = predict_cost(res_avg_cost_df)
installation_cost = predict_install_cost()

In [14]:
res_cost_prediction

,index,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2023,2024,2025,2026,2027,2028,2029,2030,Rank,Weight Score
0,HI,10.26,10.52,10.90,12.28,12.45,13.32,14.26,14.80,13.82,...,35.07,35.82,36.57,37.32,38.07,38.83,39.58,40.33,1.0,0.50
1,AK,10.11,10.67,10.82,11.15,11.32,11.24,11.36,11.44,11.50,...,25.65,26.34,27.02,27.70,28.38,29.06,29.74,30.43,2.0,1.00
2,CT,10.01,10.51,11.07,11.39,11.47,11.95,12.05,12.13,11.95,...,20.00,20.52,21.04,21.55,22.07,22.59,23.11,23.63,3.0,1.50
3,RI,9.84,10.99,11.17,11.38,11.26,11.47,11.81,6.06,9.58,...,19.48,19.77,20.06,20.35,20.64,20.94,21.23,21.52,4.0,2.00
4,MA,9.66,10.40,10.62,11.00,11.09,11.26,11.25,10.01,10.60,...,19.17,19.46,19.75,20.04,20.33,20.61,20.90,21.19,5.0,2.50
5,NH,10.34,10.38,11.36,12.31,12.91,13.50,13.44,11.05,11.09,...,17.64,17.86,18.08,18.30,18.52,18.74,18.97,19.19,6.0,3.00
6,VT,9.27,9.53,9.56,9.84,9.96,10.52,10.99,11.45,11.61,...,18.01,18.08,18.16,18.23,18.31,18.38,18.46,18.53,7.0,3.50
7,CA,9.98,10.79,11.07,11.30,11.43,11.61,11.33,5.75,9.43,...,16.74,16.94,17.15,17.35,17.56,17.76,17.97,18.17,8.0,4.00
8,ME,9.30,10.45,11.37,11.43,12.32,12.51,12.58,12.75,13.02,...,15.97,16.17,16.37,16.57,16.77,16.98,17.18,17.38,9.0,4.50
9,NY,11.44,11.97,12.43,13.17,13.55,13.90,14.04,11.28,10.96,...,16.01,16.14,16.28,16.42,16.56,16.70,16.84,16.97,10.0,5.00


In [54]:
def YoY_ranking(df):
    temp_df = pd.DataFrame([])
    temp_df['index'] = df['index']
    for col in range(1998,2030):
        temp_df[col] = df[col].rank()
    
    return temp_df

In [55]:
YoY_Installation_Ranking = YoY_ranking(installation_cost).head(10)
YoY_Energy_Sales_Ranking = YoY_ranking(res_sales_prediction).head(10)
YoY_Energy_Cost_Ranking = YoY_ranking(res_cost_prediction).head(10)

YoY_Installation_Ranking = YoY_Installation_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank")
YoY_Energy_Sales_Ranking = YoY_Energy_Sales_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank")
YoY_Energy_Cost_Ranking = YoY_Energy_Cost_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank")

YoY_Installation_Ranking = YoY_Installation_Ranking.rename({'index': 'State'}, axis=1)
YoY_Energy_Sales_Ranking = YoY_Energy_Sales_Ranking.rename({'index': 'State'}, axis=1)
YoY_Energy_Cost_Ranking = YoY_Energy_Cost_Ranking.rename({'index': 'State'}, axis=1)

In [59]:
#YoY_Installation_Wgt_Ranking = YoY_Installation_Ranking.melt(id_vars=["State"], var_name="Date", value_name="Weight Score")
#YoY_Energy_Sales_Wgt_Ranking = YoY_Energy_Sales_Ranking.melt(id_vars=["State"], var_name="Date", value_name="Weight Score")
#YoY_Energy_Cost_Wgt_Ranking = YoY_Energy_Cost_Ranking.melt(id_vars=["State"], var_name="Date", value_name="Weight Score")

#total_ranking = YoY_Installation_Wgt_Ranking.merge(YoY_Energy_Sales_Wgt_Ranking, how='left', left_on=['State','Date'], right_on=['State','Date'])
YoY_Installation_Wgt_Ranking

,index,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,NM,2.0,4.0,4.0,5.0,6.0,4.0,5.0,7.0,6.0,...,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,ME,3.0,5.0,5.0,6.0,7.0,5.0,6.0,8.0,7.0,...,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,VT,4.0,6.0,6.0,7.0,8.0,3.0,2.0,3.0,2.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,NH,5.0,7.0,7.0,9.0,9.0,6.0,9.0,10.0,8.0,...,10.0,9.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
4,TX,6.0,8.0,10.0,10.0,10.0,8.0,7.0,6.0,4.0,...,5.0,5.0,4.0,5.0,6.0,5.0,5.0,5.0,5.0,5.0
5,MD,7.0,9.0,11.0,11.0,12.0,9.0,11.0,5.0,9.0,...,8.0,8.0,6.0,6.0,7.0,6.0,6.0,6.0,6.0,6.0
6,UT,9.0,11.0,13.0,13.0,15.0,11.0,13.0,13.0,13.0,...,1.0,1.0,12.0,7.0,5.0,7.0,7.0,7.0,7.0,7.0
7,CT,8.0,10.0,12.0,12.0,14.0,10.0,12.0,11.0,12.0,...,6.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
8,CO,10.0,12.0,14.0,15.0,3.0,13.0,15.0,1.0,10.0,...,11.0,10.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
9,WI,11.0,13.0,15.0,16.0,2.0,1.0,3.0,2.0,5.0,...,9.0,11.0,10.0,11.0,10.0,10.0,10.0,10.0,10.0,10.0


In [17]:
installation_plot = alt.Chart(YoY_Installation_Ranking).mark_line().encode(
    x='Date:O',
    y='Rank',
    color='State'
).properties(
    title='Top 10 States Based on Installation Cost'
)

#states = YoY_Installation_Ranking['index'].unique()

#state_dropdown = alt.binding_select(options=states)
#state_select = alt.selection_single(fields=['State'], bind=state_dropdown, name="Select")

#filter_state = installation_plot.add_selection(
#    state_select
#).transform_filter(
#    state_select
#)

installation_plot

alt.Chart(...)

In [18]:
energy_sales_plot = alt.Chart(YoY_Energy_Sales_Ranking).mark_line().encode(
    x='Date:O',
    y='Rank',
    color='State'
).properties(
    title='Top 10 States Based on Energy Sales'
)

energy_sales_plot

alt.Chart(...)

In [19]:
energy_cost_plot = alt.Chart(YoY_Energy_Cost_Ranking).mark_line().encode(
    x='Date:O',
    y='Rank',
    color='State'
).properties(
    title='Top 10 States Based on Energy Cost'
)

energy_cost_plot

alt.Chart(...)

In [20]:
economic_df1 = res_cost_prediction.merge(installation_cost, how='left', left_on='index', right_on='index')
economic_df1 = economic_df1[['index','Rank_x','Weight Score_x','Rank_y','Weight Score_y']]
economic_df1 = economic_df1.rename({'Rank_x':'Energy Cost Rank', 'Weight Score_x':'Energy Cost Weighted Score','Rank_y':'Installation Cost Rank', 'Weight Score_y':'Installation Cost Weighted Score'}, axis=1)

economic_df = economic_df1.merge(res_sales_prediction, how='left', left_on='index', right_on='index')
economic_df = economic_df[['index','Installation Cost Rank','Installation Cost Weighted Score','Energy Cost Rank','Energy Cost Weighted Score','Rank','Weight Score']]
economic_df = economic_df.rename({'Rank':'Energy Usage Rank', 'Weight Score':'Energy Usage Weighted Score'}, axis=1)

In [21]:
economic_df['Installation Cost Rank'].fillna((economic_df['Installation Cost Rank'].quantile(0.75)), inplace=True)
economic_df['Installation Cost Weighted Score'].fillna((economic_df['Installation Cost Weighted Score'].quantile(0.75)), inplace=True)

In [22]:
economic_df['Total Rank'] = economic_df[['Installation Cost Rank', 'Energy Cost Rank', 'Energy Usage Rank']].mean(axis=1)
economic_df['Total Weighted Score'] = economic_df[['Installation Cost Weighted Score', 'Energy Cost Weighted Score', 'Energy Usage Weighted Score']].sum(axis=1)
economic_ranking_df = economic_df.sort_values('Total Weighted Score', ascending=True).round(2)

In [67]:
rank_table = economic_ranking_df[['index','Installation Cost Weighted Score','Energy Cost Weighted Score','Energy Usage Weighted Score','Total Weighted Score']]
rank_table.rename({'index': 'State'}, axis=1).head(10)

,State,Installation Cost Weighted Score,Energy Cost Weighted Score,Energy Usage Weighted Score,Total Weighted Score
5,NH,1.20,3.0,1.0,5.20
6,VT,0.90,3.5,0.8,5.20
8,ME,0.60,4.5,1.2,6.30
2,CT,2.40,1.5,2.4,6.30
1,AK,5.02,1.0,0.4,6.42
0,HI,5.02,0.5,1.6,7.12
3,RI,5.40,2.0,0.2,7.60
4,MA,6.00,2.5,4.2,12.70
19,NM,0.30,10.0,3.2,13.50
12,WV,5.02,6.5,3.0,14.52


In [41]:
fig = px.choropleth(economic_ranking_df,  # Input Pandas DataFrame
                    locations="index",  # DataFrame column with locations
                    color="Total Weighted Score",
                    color_continuous_scale="rdylgn",
                    range_color=(0, 50),# DataFrame column with color values
                    hover_name="index", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
fig.update_layout(
    title_text = 'State Rankings', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()